# 세팅

In [ ]:
# 메인 라이브러리 불러오기
import numpy as np
import pandas as pd

# 시각화 관련 라이브러리 불러오기
import matplotlib.pyplot as plt
%matplotlib inline
import plotly
import plotly.graphFp_objs as go

# PCA 코드
from sklearn.decomposition import PCA

## tsne 코드
from sklearn.manifold import TSNE

In [ ]:
# 구글 드라이브 연결하기
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/modeling/

/content/drive/.shortcut-targets-by-id/1wuSyaB4gVB_OYC11vUBFSUjLBBeITS80/modeling


# 데이터 전처리

In [ ]:
# 데이터셋 불러오기
df = pd.read_csv('data/naver_data/naver_review.csv')

In [ ]:
df.head(5)

,search_name,address,road_address,near_station,station_distance,total_review,"""커피가 맛있어요""","""친절해요""","""음료가 맛있어요""","""디저트가 맛있어요""",...,"""대화하기 좋아요""","""집중하기 좋아요""","""인테리어가 멋져요""","""좌석이 편해요""","""화장실이 깨끗해요""","""특별한 메뉴가 있어요""","""가성비가 좋아요""","""뷰가 좋아요""","""주차하기 편해요""","""사진이 잘 나와요"""
0,버블베어,방화동,서울 강서구 금낭화로 167 방화8단지개화아파트,5방화역,427.0,18,7.0,5.0,5.0,6.0,...,0.0,0.0,0.0,1.0,2.0,2.0,0.0,0.0,1.0,0.0
1,커피빈 낙성대역점,봉천동,"서울 관악구 남부순환로 1909 1,2층",2낙성대역,119.0,284,7.0,56.0,74.0,37.0,...,21.0,48.0,8.0,33.0,2.0,2.0,8.0,14.0,0.0,1.0
2,노란꼬무줄 선릉본점,대치동,서울 강남구 선릉로 424 1층5호 노란꼬무줄,2수인분당선릉역,157.0,134,40.0,14.0,61.0,39.0,...,0.0,0.0,1.0,0.0,0.0,21.0,33.0,0.0,1.0,0.0
3,커피랩,서교동,서울 마포구 와우산로29길 14,2공항경의중앙홍대입구역,324.0,100,76.0,23.0,13.0,9.0,...,17.0,10.0,19.0,2.0,6.0,10.0,4.0,5.0,0.0,4.0
4,커핀그루나루 신천점,잠실동,서울 송파구 올림픽로12길 8,2잠실새내역,228.0,140,64.0,14.0,31.0,13.0,...,29.0,7.0,2.0,15.0,5.0,5.0,6.0,3.0,0.0,1.0


In [ ]:
# 데이터에서 feature들(키워드)을 뽑아내기
cafe_review = df.drop(columns=['search_name','address','road_address','near_station','station_distance','total_review'],axis=1)

# 카페명을 list로 저장하기
cafe_name = df['search_name']
unique_text = list(cafe_name)

In [ ]:
# 데이터 스케일링
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(cafe_review.values)
review_scaled= scaler.transform(cafe_review.values)

# PCA와 TSNE 결과

pca 코드 reference: 7기 최명헌님 세션 자료

## 스케일링만 진행한 데이터 이용


In [ ]:
## pca를 2개의 component로 한 후 tsne 적용하기
pca = PCA(n_components=2)
pca_result = pca.fit_transform(review_scaled)

tsne = TSNE(n_components=2, verbose=1, learning_rate = 0.05, perplexity = 40, n_iter=1000)
tsne_result = tsne.fit_transform(pca_result)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.



[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 7661 samples in 0.007s...
[t-SNE] Computed neighbors for 7661 samples in 0.353s...
[t-SNE] Computed conditional probabilities for sample 1000 / 7661
[t-SNE] Computed conditional probabilities for sample 2000 / 7661
[t-SNE] Computed conditional probabilities for sample 3000 / 7661
[t-SNE] Computed conditional probabilities for sample 4000 / 7661
[t-SNE] Computed conditional probabilities for sample 5000 / 7661
[t-SNE] Computed conditional probabilities for sample 6000 / 7661
[t-SNE] Computed conditional probabilities for sample 7000 / 7661
[t-SNE] Computed conditional probabilities for sample 7661 / 7661
[t-SNE] Mean sigma: 0.054230
[t-SNE] KL divergence after 250 iterations with early exaggeration: 92.458939
[t-SNE] KL divergence after 650 iterations: 5.224426


In [ ]:
# tsne

data = go.Scatter(
    x=tsne_result[:,0],
    y=tsne_result[:,1],
    text = ['point #{}'.format(i) for i in cafe_name],
    mode='markers',
    marker=dict(
        size=5,
        color="green",
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

## 키워드 데이터를 투표한 사람수로 나눈 후 스케일링한 데이터 이용

In [ ]:
# 키워드 열들을 투표한 사람수로 나누기
review_p = cafe_review.div(df['total_review'], axis=0)

In [ ]:
# 결과
review_p.head()

,"""커피가 맛있어요""","""친절해요""","""음료가 맛있어요""","""디저트가 맛있어요""","""매장이 청결해요""","""대화하기 좋아요""","""집중하기 좋아요""","""인테리어가 멋져요""","""좌석이 편해요""","""화장실이 깨끗해요""","""특별한 메뉴가 있어요""","""가성비가 좋아요""","""뷰가 좋아요""","""주차하기 편해요""","""사진이 잘 나와요"""
0,0.388889,0.277778,0.277778,0.333333,0.277778,0.000000,0.000000,0.000000,0.055556,0.111111,0.111111,0.000000,0.000000,0.055556,0.000000
1,0.024648,0.197183,0.260563,0.130282,0.140845,0.073944,0.169014,0.028169,0.116197,0.007042,0.007042,0.028169,0.049296,0.000000,0.003521
2,0.298507,0.104478,0.455224,0.291045,0.014925,0.000000,0.000000,0.007463,0.000000,0.000000,0.156716,0.246269,0.000000,0.007463,0.000000
3,0.760000,0.230000,0.130000,0.090000,0.080000,0.170000,0.100000,0.190000,0.020000,0.060000,0.100000,0.040000,0.050000,0.000000,0.040000
4,0.457143,0.100000,0.221429,0.092857,0.107143,0.207143,0.050000,0.014286,0.107143,0.035714,0.035714,0.042857,0.021429,0.000000,0.007143


In [ ]:
# 전체 리뷰한 사람수 열도 추가하기
review_p['total_review'] = df['total_review']

In [ ]:
review_p.head()

,"""커피가 맛있어요""","""친절해요""","""음료가 맛있어요""","""디저트가 맛있어요""","""매장이 청결해요""","""대화하기 좋아요""","""집중하기 좋아요""","""인테리어가 멋져요""","""좌석이 편해요""","""화장실이 깨끗해요""","""특별한 메뉴가 있어요""","""가성비가 좋아요""","""뷰가 좋아요""","""주차하기 편해요""","""사진이 잘 나와요""",total_review
0,0.388889,0.277778,0.277778,0.333333,0.277778,0.000000,0.000000,0.000000,0.055556,0.111111,0.111111,0.000000,0.000000,0.055556,0.000000,18
1,0.024648,0.197183,0.260563,0.130282,0.140845,0.073944,0.169014,0.028169,0.116197,0.007042,0.007042,0.028169,0.049296,0.000000,0.003521,284
2,0.298507,0.104478,0.455224,0.291045,0.014925,0.000000,0.000000,0.007463,0.000000,0.000000,0.156716,0.246269,0.000000,0.007463,0.000000,134
3,0.760000,0.230000,0.130000,0.090000,0.080000,0.170000,0.100000,0.190000,0.020000,0.060000,0.100000,0.040000,0.050000,0.000000,0.040000,100
4,0.457143,0.100000,0.221429,0.092857,0.107143,0.207143,0.050000,0.014286,0.107143,0.035714,0.035714,0.042857,0.021429,0.000000,0.007143,140


In [ ]:
# 데이터 스케일링

scaler = StandardScaler()
scaler.fit(review_p.values)
review_p= scaler.transform(review_p.values)

In [ ]:
## pca를 2개의 component로 한 후 tsne 적용하기
pca = PCA(n_components=2)
pca_result = pca.fit_transform(review_p)

tsne = TSNE(n_components=2, verbose=1, learning_rate = 0.05, perplexity = 40, n_iter=1000)
tsne_result = tsne.fit_transform(pca_result)

In [ ]:
# tsne

data = go.Scatter(
    x=tsne_result[:,0],
    y=tsne_result[:,1],
    text = ['point #{}'.format(i) for i in cafe_name],
    mode='markers',
    marker=dict(
        size=5,
        color ='green',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()